In [3]:
import os
import shutil
from PIL import Image
import random

# Фиксируем random seed для воспроизводимости
random.seed(228)

# Конфигурация
dataset_path = "dataset"
images_dir = os.path.join(dataset_path, "images")
labels_dir = os.path.join(dataset_path, "labels")

# Создаем папки для train/val/test
for split in ['train', 'val', 'test']:
    os.makedirs(f'images/{split}', exist_ok=True)
    os.makedirs(f'labels/{split}', exist_ok=True)

# Функция для поиска изображения по имени (без учета расширения)
def find_image_file(base_name):
    """Находит файл изображения по базовому имени (без учета расширения)"""
    for file in os.listdir(images_dir):
        file_base = os.path.splitext(file)[0]
        if file_base == base_name:
            return file
    return None

# Находим все валидные пары
valid_pairs = []
all_label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]

print("Поиск валидных пар изображение-разметка...")
for label_file in all_label_files:
    base_name = os.path.splitext(label_file)[0]
    image_file = find_image_file(base_name)
    
    if image_file:
        valid_pairs.append((image_file, label_file))
        print(f"✓ Найдена пара: {image_file} -> {label_file}")
    else:
        print(f"✗ Не найдено изображение для: {label_file}")

print(f"\nВсего валидных пар: {len(valid_pairs)}")

if len(valid_pairs) == 0:
    print("Ошибка: не найдено ни одной валидной пары!")
    exit()

# Перемешиваем пары
random.shuffle(valid_pairs)

# Простое разделение (70% train, 15% val, 15% test)
train_count = int(0.7 * len(valid_pairs))
val_count = int(0.15 * len(valid_pairs))

train_pairs = valid_pairs[:train_count]
val_pairs = valid_pairs[train_count:train_count + val_count]
test_pairs = valid_pairs[train_count + val_count:]

print(f"\nРазделение:")
print(f"Train: {len(train_pairs)} изображений")
print(f"Val: {len(val_pairs)} изображений")
print(f"Test: {len(test_pairs)} изображений")

# Функция для конвертации изображений в JPG
def convert_to_jpg(source_path, target_path):
    """Конвертирует изображение в JPG формат"""
    try:
        with Image.open(source_path) as img:
            # Конвертируем в RGB если нужно (для PNG с прозрачностью)
            if img.mode in ('RGBA', 'LA', 'P'):
                background = Image.new('RGB', img.size, (255, 255, 255))
                if img.mode == 'P':
                    img = img.convert('RGBA')
                background.paste(img, mask=img.split()[-1] if img.mode == 'RGBA' else None)
                img = background
            elif img.mode != 'RGB':
                img = img.convert('RGB')
            
            img.save(target_path, 'JPEG', quality=95)
        return True
    except Exception as e:
        print(f"Ошибка конвертации {source_path}: {e}")
        return False

# Переименовываем и копируем файлы
def organize_and_copy_files(pairs, split_name, start_index):
    """Переименовывает файлы и копирует в организованную структуру"""
    split_images_dir = os.path.join('images', split_name)
    split_labels_dir = os.path.join('labels', split_name)
    
    for i, (img_file, lbl_file) in enumerate(pairs, start_index):
        # Новые имена
        new_base_name = f"image_{i:03d}"
        new_img_name = f"{new_base_name}.jpg"
        new_lbl_name = f"{new_base_name}.txt"
        
        # Конвертируем и копируем изображение
        source_img_path = os.path.join(images_dir, img_file)
        target_img_path = os.path.join(split_images_dir, new_img_name)
        
        if convert_to_jpg(source_img_path, target_img_path):
            print(f"✅ {split_name}: {img_file} -> {new_img_name}")
        else:
            # Если конвертация не удалась, просто копируем
            shutil.copy2(source_img_path, target_img_path)
            print(f"⚠️  {split_name}: {img_file} -> {new_img_name} (скопировано без конвертации)")
        
        # Копируем разметку
        source_lbl_path = os.path.join(labels_dir, lbl_file)
        target_lbl_path = os.path.join(split_labels_dir, new_lbl_name)
        shutil.copy2(source_lbl_path, target_lbl_path)
    
    return start_index + len(pairs)

# Организуем файлы по сетами
print("\nОрганизация файлов...")
next_index = 1
next_index = organize_and_copy_files(train_pairs, 'train', next_index)
next_index = organize_and_copy_files(val_pairs, 'val', next_index)
next_index = organize_and_copy_files(test_pairs, 'test', next_index)

# Удаляем лишние файлы разметки (для которых нет изображений)
print("\nОчистка лишних файлов разметки...")
deleted_count = 0
for label_file in os.listdir(labels_dir):
    if label_file.endswith('.txt'):
        base_name = os.path.splitext(label_file)[0]
        if not find_image_file(base_name):
            os.remove(os.path.join(labels_dir, label_file))
            print(f"🗑️  Удален: {label_file}")
            deleted_count += 1

# Функция для проверки распределения классов
def check_class_distribution(split_name):
    """Проверяет распределение классов в сете"""
    class_counts = {}
    split_labels_dir = os.path.join('labels', split_name)
    
    for label_file in os.listdir(split_labels_dir):
        if label_file.endswith('.txt'):
            label_path = os.path.join(split_labels_dir, label_file)
            try:
                with open(label_path, 'r', encoding='utf-8') as f:
                    for line in f:
                        parts = line.strip().split()
                        if parts:
                            class_id = int(parts[0])
                            class_counts[class_id] = class_counts.get(class_id, 0) + 1
            except Exception as e:
                print(f"Ошибка чтения {label_path}: {e}")
    
    print(f"\n{split_name.upper()} - распределение классов:")
    for class_id in sorted(class_counts.keys()):
        print(f"  Класс {class_id}: {class_counts[class_id]} экземпляров")
    
    return class_counts

print(f"\n=== ПРОВЕРКА РАСПРЕДЕЛЕНИЯ КЛАССОВ ===")
train_dist = check_class_distribution('train')
val_dist = check_class_distribution('val')
test_dist = check_class_distribution('test')

print(f"\n=== ГОТОВО ===")
print(f"Удалено лишних файлов разметки: {deleted_count}")
print(f"Всего оставлено пар: {len(valid_pairs)}")
print(f"Train: {len(train_pairs)}")
print(f"Val: {len(val_pairs)}")
print(f"Test: {len(test_pairs)}")
print("\nСтруктура папок:")
print("images/train/    image_001.jpg, image_002.jpg, ...")
print("images/val/      image_001.jpg, image_002.jpg, ...")
print("images/test/     image_001.jpg, image_002.jpg, ...")
print("labels/train/    image_001.txt, image_002.txt, ...")
print("labels/val/      image_001.txt, image_002.txt, ...")
print("labels/test/     image_001.txt, image_002.txt, ...")

Поиск валидных пар изображение-разметка...
✓ Найдена пара: 018aaf0f-7558-2.jpg -> 018aaf0f-7558-2.txt
✓ Найдена пара: 01b0cc79-photo_2019-11-16_14-52-12_wp1d-uu.jpg -> 01b0cc79-photo_2019-11-16_14-52-12_wp1d-uu.txt
✓ Найдена пара: 021c5251-5d466c7e252a2df1b6d95f8a95cb6af5.jpeg -> 021c5251-5d466c7e252a2df1b6d95f8a95cb6af5.txt
✓ Найдена пара: 02a250ee-Image_68.jpg -> 02a250ee-Image_68.txt
✓ Найдена пара: 03274eea-Image_26.jpg -> 03274eea-Image_26.txt
✓ Найдена пара: 03ac4fb3-Image_24.jpg -> 03ac4fb3-Image_24.txt
✓ Найдена пара: 03bcba67-niF8Mv2bWQ8.jpg -> 03bcba67-niF8Mv2bWQ8.txt
✓ Найдена пара: 03f9329c-pq6h7ost2kav9cuer1lf1wg2ss63ntww.jpg -> 03f9329c-pq6h7ost2kav9cuer1lf1wg2ss63ntww.txt
✓ Найдена пара: 04e79e8d-Image_79.jpg -> 04e79e8d-Image_79.txt
✗ Не найдено изображение для: 052cb597-_%D1%84%D0%BE%D1%82%D0%BE_10_0x200_2b7.txt
✓ Найдена пара: 06265a83-Image_19.jpg -> 06265a83-Image_19.txt
✓ Найдена пара: 066d8a89-_.jpg -> 066d8a89-_.txt
✓ Найдена пара: 06799cf4-bw_55_park_1.jpg -> 06

In [1]:
!pip install --upgrade "requests>=2.32.3,<2.33.0"

# Обновление urllib3
!pip install --upgrade "urllib3>=2.5.0"

# Обновление других пакетов
!pip install --upgrade pytz setuptools

# Установка YOLO
!pip install ultralytics


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from ultralytics import YOLO

model = YOLO('yolo11m.pt')  # Загрузка предобученной модели YOLOv8m

results = model.train(data='dataset/data.yaml', epochs=40, imgsz=480, batch=16, patience=10, name='equipment_detection_experiment')

New https://pypi.org/project/ultralytics/8.3.229 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.228  Python-3.13.7 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 5070, 12227MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=480, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=equipment_detection_experiment, nbs

In [4]:
from ultralytics import YOLO

model = YOLO('runs/detect/equipment_detection_experiment/weights/best.pt')

# Валидация с сохранением confusion matrix
results = model.val(
    data='dataset/data.yaml',
    split='test',
    save_json=True,
    plots=True  # Это сохранит confusion matrix
)

print("Confusion matrix сохранена в runs/detect/val/confusion_matrix_test.png")

Ultralytics 8.3.228  Python-3.13.7 torch-2.8.0+cu129 CUDA:0 (NVIDIA GeForce RTX 5070, 12227MiB)
YOLO11m summary (fused): 125 layers, 20,039,284 parameters, 0 gradients, 67.7 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 518.6263.4 MB/s, size: 369.3 KB)
val: Scanning C:\Study\Masters\term_3\CV-Workout-Tracker\equipment_detection\dataset\labels\test.cache... 105 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 105/105 180.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 1.6it/s 4.4s0.3ss
                   all        105        410      0.708      0.494      0.571      0.328
  _,         43         61      0.759      0.639      0.728      0.474
_,         12         12      0.562      0.322      0.237      0.155
__,          8         16      0.386      0.125      0.208     0.0948
__,         50        175      0.588      0.269      0.357      0.195
             _,         29         37      0.954     

In [5]:
print(" Результаты на test данных:")
print(f"mAP50: {results.box.map50:.3f}")
print(f"mAP50-95: {results.box.map:.3f}")

# Правильное извлечение Precision и Recall
print(f"Precision: {results.box.p.mean():.3f}")  #Среднее по всем классам
print(f"Recall: {results.box.r.mean():.3f}")     #Среднее по всем классам

# Детальная информация по классам
print(f"\nВсего классов: {len(results.box.ap)}")
print(f"Классы: {list(model.names.values())}")

 Результаты на test данных:
mAP50: 0.571
mAP50-95: 0.328
Precision: 0.708
Recall: 0.494

Всего классов: 11
Классы: ['Брусья_параллельные,', 'Кольца_гимнастические,', 'Перекладина_для_отжиманий,', 'Перекладина_для_подтягиваний,', 'Рукоход_,', 'Рукоход_змеевик,', 'Скамья_для_пресса_наклонная,', 'Скамья_для_отжиманий,', 'Скамья_для_пресса,', 'Тренажер_для_пресса_с_упором,', 'Тумба_для_запрыгивания,', 'шведская_стенка']
